In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('input/2012/qb/weeks1_8.csv')
df1=pd.read_csv('input/2012/qb/weeks_9_16.csv')
df3 = pd.read_csv('input/2012/qb/weeks_17.csv')

In [3]:
raw_qb_2012 = pd.concat([df,df1], axis=0)
raw_qb_2012 = pd.concat([raw_qb_2012, df3], axis=0)
raw_qb_2012.drop('Rk', axis =1, inplace = True)
raw_qb_2012.shape

(629, 18)

In [4]:
raw_qb_2012.head()

,Player,Pos,Week,Team,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,QB Rating,Att.1,Yds.1,Yds/Att.1,TD.1,Fantasy Points
0,Matt Ryan,QB,1,ATL,KC,23,31,74,299,10,3,0,136,3,25,8,1,32.46
1,Robert Griffin III,QB,1,WAS,NO,19,26,73,320,12,2,0,140,9,42,5,0,25.00
2,Tony Romo,QB,1,DAL,NYG,22,29,76,307,11,3,1,130,5,12,2,0,23.48
3,Aaron Rodgers,QB,1,GB,SF,30,44,68,303,7,2,1,93,5,27,5,0,22.82
4,Drew Brees,QB,1,NO,WAS,24,52,46,339,7,3,2,71,0,0,0,0,21.56


In [5]:
raw_qb_2012 = raw_qb_2012.rename(columns={'Att.1':'rush_att',
                       'Yds.1':'rush_yds',
                       'Yds/Att.1':'rush_yd/att',
                       'TD.1':'rush_td'})

In [6]:
team_stats = raw_qb_2012.groupby(['Player','Week','Opp']).sum()

In [7]:
team_stats.head(), team_stats.shape

(                        Comp  Att  Pct  Yds  Yds/Att  TD  Int  QB Rating  \
 Player        Week Opp                                                     
 Aaron Rodgers 1    SF     30   44   68  303        7   2    1         93   
               2    CHI    22   32   69  219        7   1    1         85   
               3    SEA    26   39   67  223        6   0    0         81   
               4    NO     31   41   76  319        8   4    1        120   
               5    IND    20   32   63  235        7   3    1        103   
 
                         rush_att  rush_yds  rush_yd/att  rush_td  \
 Player        Week Opp                                             
 Aaron Rodgers 1    SF          5        27            5        0   
               2    CHI         3        -6           -2        0   
               3    SEA         2        17            9        0   
               4    NO          5        13            3        0   
               5    IND         5        57  

In [9]:
# Define a dataframe to contain the current year 3 game moving averages of all 2012 QBs
def create_recent_dataframe(df):
    columns = []
    for col in df.columns:
        columns.append('{}_ma'.format(col.lower()))
    new_df = pd.DataFrame(columns=columns, index =df.index)
    return new_df

In [10]:
# Create recent and current dataframes for all 2012 QBs
qb_recent_2012= create_recent_dataframe(team_stats)

In [14]:
#Populate recent dataframe with 3 game moving average for all qb's in 2012 season
# DO NOT CHANGE !!!!!!!!
for col in team_stats:  # Iterate through every attribute in the team_stats dataframe
    ma3 = []  # Create an empty list to hold the season average of the current attribute
    for player in team_stats.index.levels[0]:  # Iterate through every player in the team_stats df
        total = [] # INstantiate a deque container with a max length of 3 to hold the three most current games for player
        count = 0 # Counter to track week number 
        for week in team_stats.loc[player].index: # Iterate through every week for relevant player d
            if count < 3: # check count value to start 3 game moving average
                ma3.append(0)  # week 1 through 3 have 3 game movong average of 0
                total.append(team_stats.loc[player].loc[week][col])  # Add the value current player's attribute total from the left
                count += 1  # Increase count
            else:  # Once 3 game moving average is avaliable
                ma3.append(np.mean(total))  # Add the average of the 3 games held in totals 3 game moivng average list
                total.append(team_stats.loc[player].loc[week][col]) # Replace 3 set block with a first in first out process
    qb_recent_2012['{}_ma'.format(col.lower())] = ma3  # Populate databse with 3 game moving average list

In [15]:
qb_recent_2012.shape

(629, 13)

In [16]:
qb_recent_2012[0:100]

comp_ma     att_ma     pct_ma      yds_ma  \
Player             Week Opp                                                
Aaron Rodgers      1    SF    0.000000   0.000000   0.000000    0.000000   
                   2    CHI   0.000000   0.000000   0.000000    0.000000   
                   3    SEA   0.000000   0.000000   0.000000    0.000000   
                   4    NO   26.000000  38.333333  68.000000  248.333333   
                   5    IND  27.250000  39.000000  70.000000  266.000000   
                   6    HOU  25.800000  37.600000  68.600000  259.800000   
                   7    LAR  25.500000  37.500000  68.000000  272.833333   
                   8    JAX  26.142857  37.428571  69.857143  282.714286   
                   9    ARI  25.625000  37.125000  69.000000  270.625000   
                   11   DET  24.333333  36.333333  66.555556  264.777778   
                   12   NYG  23.800000  35.400000  66.900000  261.900000   
                   13   MIN  22.909091  34.454545  65.909091  258.000000   
                   14   DET  23.250000  34.500000  66.833333  260.333333   
                   15   CHI  22.538462  33.692308  66.153846  253.615385   
                   16   TEN  22.571429  33.857143  66.000000  256.285714   
                   17   MIN  22.866667  34.133333  66.333333  262.000000   
Alex Smith         1    GB    0.000000   0.000000   0.000000    0.000000   
                   2    DET   0.000000   0.000000   0.000000    0.000000   
                   3    MIN   0.000000   0.000000   0.000000    0.000000   
                   4    NYJ  21.333333  30.666667  70.333333  213.666667   
                   5    BUF  19.000000  28.250000  67.000000  196.000000   
                   6    NYG  18.800000  27.400000  68.600000  217.400000   
                   7    SEA  18.833333  27.833333  67.666667  214.500000   
                   8    ARI  18.142857  27.142857  66.714286  203.857143   
                   10   LAR  18.125000  26.125000  70.250000  207.375000   
                   11   CHI  16.888889  24.111111  72.222222  192.333333   
                   12   NO   15.200000  21.700000  65.000000  173.100000   
                   13   LAR  13.818182  19.727273  59.090909  157.363636   
                   14   MIA  12.666667  18.083333  54.166667  144.250000   
                   15   NE   11.692308  16.692308  50.000000  133.153846   
...                                ...        ...        ...         ...   
Ben Roethlisberger 8    WAS  25.833333  39.166667  66.166667  294.166667   
                   9    NYG  25.571429  38.285714  67.142857  283.857143   
                   10   KC   25.000000  37.250000  67.500000  275.375000   
                   14   LAC  23.222222  35.111111  65.555556  254.111111   
                   15   DAL  23.100000  35.800000  64.200000  257.200000   
                   16   CIN  23.181818  36.181818  63.818182  264.636364   
                   17   CLE  22.416667  35.500000  62.666667  260.916667   
Blaine Gabbert     1    MIN   0.000000   0.000000   0.000000    0.000000   
                   2    HOU   0.000000   0.000000   0.000000    0.000000   
                   3    IND   0.000000   0.000000   0.000000    0.000000   
                   4    CIN  13.333333  26.333333  48.000000  156.000000   
                   5    CHI  15.750000  28.250000  53.000000  163.500000   
                   7    OAK  16.000000  29.200000  52.800000  159.200000   
                   8    GB   14.666667  26.333333  55.166667  151.000000   
                   9    DET  16.428571  29.571429  55.142857  172.714286   
                   10   IND  17.750000  30.625000  57.125000  178.625000   
                   11   HOU  17.777778  30.666667  57.222222  182.000000   
Brady Quinn        5    BAL   0.000000   0.000000   0.000000    0.000000   
                   6    TB    0.000000   0.000000   0.000000    0.000000   
                   8    OAK   0.000000   0.000000   0.000000    0.000000   
   

In [17]:
qb_recent_2012.head()

comp_ma     att_ma  pct_ma      yds_ma  yds/att_ma  \
Player        Week Opp                                                       
Aaron Rodgers 1    SF      0.00   0.000000     0.0    0.000000    0.000000   
              2    CHI     0.00   0.000000     0.0    0.000000    0.000000   
              3    SEA     0.00   0.000000     0.0    0.000000    0.000000   
              4    NO     26.00  38.333333    68.0  248.333333    6.666667   
              5    IND    27.25  39.000000    70.0  266.000000    7.000000   

                        td_ma    int_ma  qb rating_ma  rush_att_ma  \
Player        Week Opp                                               
Aaron Rodgers 1    SF    0.00  0.000000      0.000000     0.000000   
              2    CHI   0.00  0.000000      0.000000     0.000000   
              3    SEA   0.00  0.000000      0.000000     0.000000   
              4    NO    1.00  0.666667     86.333333     3.333333   
              5    IND   1.75  0.750000     94.750000     3.750000   

                        rush_yds_ma  rush_yd/att_ma  rush_td_ma  \
Player        Week Opp                                            
Aaron Rodgers 1    SF      0.000000            0.00         0.0   
              2    CHI     0.000000            0.00         0.0   
              3    SEA     0.000000            0.00         0.0   
              4    NO     12.666667            4.00         0.0   
              5    IND    12.750000            3.75         0.0   

                        fantasy points_ma  
Player        Week Opp                     
Aaron Rodgers 1    SF            0.000000  
              2    CHI           0.000000  
              3    SEA           0.000000  
              4    NO           14.533333  
              5    IND          17.915000

In [18]:
qb_recent_2012.tail()

comp_ma  att_ma     pct_ma  yds_ma  yds/att_ma  td_ma  \
Player       Week Opp                                                           
Tyrod Taylor 4    CLE  0.000000    0.00   0.000000     0.0    0.000000    0.0   
             5    KC   0.000000    0.00   0.000000     0.0    0.000000    0.0   
             9    CLE  0.666667    1.00  22.333333    10.0    3.333333    0.0   
             10   OAK  0.500000    0.75  16.750000     7.5    2.500000    0.0   
             17   CIN  0.400000    0.80  13.400000     6.0    2.000000    0.0   

                       int_ma  qb rating_ma  rush_att_ma  rush_yds_ma  \
Player       Week Opp                                                   
Tyrod Taylor 4    CLE     0.0          0.00     0.000000     0.000000   
             5    KC      0.0          0.00     0.000000     0.000000   
             9    CLE     0.0         33.00     0.333333     2.333333   
             10   OAK     0.0         24.75     0.500000     2.750000   
             17   CIN     0.0         27.80     0.400000     2.200000   

                       rush_yd/att_ma  rush_td_ma  fantasy points_ma  
Player       Week Opp                                                 
Tyrod Taylor 4    CLE        0.000000         0.0           0.000000  
             5    KC         0.000000         0.0           0.000000  
             9    CLE        2.333333         0.0           0.633333  
             10   OAK        2.750000         0.0           0.575000  
             17   CIN        2.200000         0.0           0.460000

In [19]:
qb_recent_2012.to_csv('recent_qb_2012.csv', index=True)